julie.tsitron@parks.nyc.gov 2/20/2020

# Imports and Connections to DBs

In [1]:
pwd

'C:\\Projects\\Structures\\scripts\\weekly_pre_QA'

In [2]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

sys.path.append('../')
from IPM_Shared_Code.Python.geo_functions import read_geosql
from IPM_Shared_Code.Python.utils import get_config
from IPM_Shared_Code.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *

ModuleNotFoundError: No module named 'IPM_Shared_Code'

In [ ]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [ ]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [ ]:
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                           ';Database=;Trusted_Connection=Yes;')

## Deal with Dates

### m/d/Y H:M:S format:

In [ ]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['COMMISSIONDATE']

In [ ]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [ ]:
# structures_dev.columns.values

# Delta Table from structuresdb

In [ ]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [ ]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [ ]:
struct_deltas.head()

In [ ]:
multipolygons = []

for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
        multipolygons.append(row['SYSTEM'])

if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons)
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
    
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

## Write Delta Table to geojson file

In [ ]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\Structures\\delta_snapshots/' + today

In [ ]:
if not struct_deltas.empty:
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

## Read geojson file to geojson object 

In [ ]:
if not struct_deltas.empty:
    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

## Create arcgis featureSet from geojson object

In [ ]:
if not struct_deltas.empty:
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

# Connect to published datasets via GIS object

In [ ]:
gis = GIS(url=portal)

# Make Edits

## Structures

### run delta archive stored proc

In [ ]:
crsr = con.cursor()
sqal_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

### Connect to structures feature layer directly

In [ ]:
strct_lyr_url = structures_url

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [ ]:
structures_features

In [ ]:
len(struct_deltas)

### EDIT

In [ ]:
if not struct_deltas.empty:
    num_updates = len(fromJSON_deltas.features)

In [ ]:
if not struct_deltas.empty:
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_structures_mssg.txt', subject = 'structures push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

## Geosupport

### connect to geosupport service

In [ ]:
geosupport_tbl_url = geosupport_url

In [ ]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

### Get latest BINs from structures layer

In [ ]:
structures_valid_BINs = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))] 

In [ ]:
bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int) %
                             100000 != 0]['BIN'].astype(int).to_list()

In [ ]:
bins = [str(bins[i]) for i in range(0, len(bins))]

### run master geosupport function

In [ ]:
df = master_geosupport_func(bins)

### prepare latest geosupport data to be pushed to prod

In [ ]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               'out_sanborn_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_com_dist': 'Community_Board',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'Sanitation Collection Scheduling Section and Subsection': 'Sanitation_Subsect',
                               'Sanitation District': 'Sanitation_District',
                               'Sanitation Recycling Collection Schedule': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID',
                               'out_ap_id': 'Address_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup', 'Address_ID'])
       .drop_duplicates())

In [ ]:
len(dff)

In [ ]:
dff.head(10)

In [ ]:
measurer = np.vectorize(len)

In [ ]:
# type_df = pd.DataFrame({'df_name': dff.columns.tolist(),
#                         'df_type': dff.dtypes.astype(str).tolist(),
#                         'df_len' : measurer(dff.values.astype(str)).max(axis=0)})

In [ ]:
# type_df.head()

In [ ]:
con_str = "Driver={SQL Server};Server=" + server + ";Database=structuresdb;Trusted_Connection=Yes;"
sa_con = quote_plus(con_str)
engine = sqlalchemy.create_engine(
    "mssql+pyodbc:///?odbc_connect={}".format(sa_con))

In [ ]:
if len(dff) > 0:
    dff.to_sql('tbl_geosupport_address',
               engine,
               schema='dbo',
               if_exists='replace',
               index=False)

In [ ]:
dff.Boro_Code.fillna(0,inplace=True)

In [ ]:
geosupport_FSET = FeatureSet.from_dataframe(dff)

In [ ]:
len(geosupport_FSET.features)

### EDIT

In [ ]:
if len(dff) > 0: #len(geosupport_FSET.features):
    try:
        ## TRUNCATE AND REPLACE
        tbl_geosupport.delete_features(where="objectid > 0")
        add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)
    except:
        error_mssg = 'Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(len(dff)) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_geosupport_mssg.txt', subject = 'geosupport push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the geosupport table.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport: NO UPDATES to push', e=mssg)